<h1 align="center">Environmental Sound Classification</h1>

## Introduction

Environmental Sound Classification: Develop a Python-based program that can classify different types of environmental sounds. The program should take audio files as input and predict the sound category (e.g., dog bark, car horn, rain, footsteps). The output should include the predicted class label and confidence score for each input sound. Optionally, visualize the waveform and spectrogram for better interpretability.

### Dataset used: 

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, children_playing, dog_bark, drilling, enginge_idling, gun_shot, jackhammer, siren, and street_music. The classes are drawn from the urban sound taxonomy. For a detailed description of the dataset and how it was compiled please refer to our paper.
All excerpts are taken from field recordings uploaded to www.freesound.org. The files are pre-sorted into ten folds (folders named fold1-fold10) to help in the reproduction of and comparison with the automatic classification results reported in the article above.
In addition to the sound excerpts, a CSV file containing metadata about each excerpt is also provided.

### Imports:

In [1]:
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

### Configs / Path Initialisations

In [2]:
audio_path= r"E:\Me\coding\jupyter\Environmental_Sound_Classification\dataset\UrbanSound8K\audio"
metadata_path= r"E:\Me\coding\jupyter\Environmental_Sound_Classification\dataset\UrbanSound8K\metadata\UrbanSound8K.csv"

In [3]:
OUTPUT_PATH = "fold_features"
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [4]:
metadata_df= pd.read_csv(metadata_path, usecols=["slice_file_name", "fold", "classID"],dtype={"fold": "uint8", "classID" : "uint8"})
metadata_df

,slice_file_name,fold,classID
0,100032-3-0-0.wav,5,3
1,100263-2-0-117.wav,5,2
2,100263-2-0-121.wav,5,2
3,100263-2-0-126.wav,5,2
4,100263-2-0-137.wav,5,2
...,...,...,...
8727,99812-1-2-0.wav,7,1
8728,99812-1-3-0.wav,7,1
8729,99812-1-4-0.wav,7,1
8730,99812-1-5-0.wav,7,1


### Data-Preprocessing

#### Using Mel-Frequency Cepstral Coefficients from Librosa:

MFCC stands for Mel-frequency Cepstral Coefficients. It’s a feature used in automatic speech and speaker recognition. Essentially, it’s a way to represent the short-term power spectrum of a sound which helps machines understand and process human speech more effectively. Imagine your voice as a unique fingerprint. MFCCs, function similarly to a unique code capturing the salient features of your speech and enabling computers to discern between distinct words, and sounds. In speech recognition applications where computers must translate spoken words into text this code is especially helpful.[4]

In [5]:
def extract_mfcc(file_path, sr=22050, n_mfcc=120, max_len=173):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)  # Standardization
        
        # Fix length (pad or truncate to max_len time steps)
        if mfcc.shape[1] < max_len:
            pad_width = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_len]
        
        # Flatten to 1D vector
        return mfcc.flatten()
    except Exception as e:
        print("Error processing", file_path, e)
        return None


#### Using predefined 10-Fold as guided in the dataset [1]

To ensure valid and comparable results when using the UrbanSound8K dataset, it is essential to follow the official 10-fold cross-validation protocol without reshuffling the data. Using the predefined folds prevents data leakage caused by placing related samples in both training and testing sets, which can falsely inflate model performance. Additionally, evaluating on all 10 folds (not just one) and averaging the results is crucial, as individual folds vary in difficulty. This approach aligns your results with prior research and ensures accurate performance assessment.

In [8]:
# Load metadata
metadata = pd.read_csv(metadata_path)

# Process each fold
for fold in range(1, 11):
    fold_df = metadata[metadata['fold'] == fold]
    X, y = [], []

    print(f"Processing Fold {fold}...")

    for _, row in tqdm(fold_df.iterrows(), total=len(fold_df)):
        filepath = os.path.join(audio_path, f"fold{fold}", row["slice_file_name"])
        label = row["classID"]

        features = extract_mfcc(filepath)
        if features is not None:
            X.append(features)
            y.append(label)

    X = np.array(X)
    y = np.array(y)

    np.save(os.path.join(OUTPUT_PATH, f"X_fold{fold}.npy"), X)
    np.save(os.path.join(OUTPUT_PATH, f"y_fold{fold}.npy"), y)

print("✅ Preprocessing completed and saved fold-wise.")

Processing Fold 1...


 94%|█████████▍| 824/873 [00:09<00:00, 127.42it/s]C:\Users\krjig\AppData\Roaming\Python\Python310\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\krjig\AppData\Roaming\Python\Python310\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
C:\Users\krjig\AppData\Roaming\Python\Python310\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
100%|██████████| 873/873 [00:10<00:00, 85.08it/s] 


Processing Fold 2...


100%|██████████| 888/888 [00:07<00:00, 119.91it/s]


Processing Fold 3...


...

Processing Fold 9...


100%|██████████| 816/816 [00:07<00:00, 113.44it/s]


Processing Fold 10...


100%|██████████| 837/837 [00:07<00:00, 114.51it/s]

✅ Preprocessing completed and saved fold-wise.


### References:

1. J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.
2. Sanjoy Barua, Tahmina Akter, Mahmud Abu Saleh Musa, Muhammad Anwarul Azim . A Deep Learning Approach for Urban Sound Classification. International Journal of Computer Applications. 185, 24 ( Jul 2023), 8-14. DOI=10.5120/ijca2023922991
3. https://www.youtube.com/watch?v=mHPpCXqQd7Y
4. https://www.geeksforgeeks.org/nlp/mel-frequency-cepstral-coefficients-mfcc-for-speech-recognition/